In [1]:
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-ja2kghcd/kobart_6afcd913a3914e4289c8411c7f32a54e
  Running command git clone --filter=blob:none -q https://github.com/SKT-AI/KoBART /tmp/pip-install-ja2kghcd/kobart_6afcd913a3914e4289c8411c7f32a54e
  Resolved https://github.com/SKT-AI/KoBART to commit 0636edf20390cbc72e8f676c6e8aeee086fd4f95
  Preparing metadata (setup.py) ... done


In [2]:
import os

from transformers import BartModel, BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer

import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from tqdm import tqdm

In [4]:
tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

using cached model
using cached model


In [5]:
print(tokenizer)
print(model)

PreTrainedTokenizerFast(name_or_path='', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'})
BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,

In [7]:
tokenizer.convert_ids_to_tokens([0, 1, 2, 3, 4, 5,])

['<s>', '</s>', '<usr>', '<pad>', '<sys>', '<unk>']

In [40]:
test_text = '''한국 전쟁 직후 현대건설은 전쟁으로 파괴된 도시와 교량, 도로, 집, 건물 등을 복구하면서 점차 늘어가는 건설수요로 승승장구하게 되었다. 그 뒤에도 늘어나는 건설 수요 등을 감안하여 그는 시멘트 공장 설립을 추진, 1964년 6월 현대 시멘트공장을 준공하여 시멘트도 자체적으로 조달하였다. 그 뒤 낙동강 고령교 복구, 한강 인도교 복구, 제1한강교 복구, 인천 제1도크 복구 등의 사업을 수주하여 1960년에는 국내 건설업체중 도급한도액이 1위를 차지하게 되었다. 1964년 단양에 시멘트 공장을 완공하였으며, 1965년에는 국내 최초로 태국의 파타니 나라티왓 고속도로를 건설하였다. 1967년에는 다시 자동차 산업에 뛰어들어 현대자동차주식회사를 설립하였다.\n\n현대건설 내 시멘트공장을 확장하여 1970년 1월 정식으로 현대시멘트주식회사를 설립하였다. 이후 현대건설과 현대시멘트의 사주로 해외건설시장 확보와 낙찰 등을 이끌어내며 한국 국외의 건설시장으로도 진출하였고 울산 조선소 건설, 서산 앞바다 간척사업 등을 성공적으로 추진하면서 기업을 확장하게 된다.\n\n1971년 1월 현대자동차, 현대건설, 현대시멘트주식회사 등을 총괄한 현대그룹을 창립하고 대표이사 회장에 취임하였다. 1973년 12월에는 중공업에도 진출하였다.
'''
inputs = tokenizer([test_text], return_tensors='pt')

In [41]:
encoder = model.get_encoder()
print(encoder)

BartEncoder(
  (embed_tokens): Embedding(30000, 768, padding_idx=3)
  (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
  (layers): ModuleList(
    (0): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (1): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=76

In [42]:
out = encoder(inputs['input_ids'])
print(out[0].shape)
# 1, 261, 768

torch.Size([1, 260, 768])


In [43]:
decoder = model.get_decoder()
print(decoder)

BartDecoder(
  (embed_tokens): Embedding(30000, 768, padding_idx=3)
  (embed_positions): BartLearnedPositionalEmbedding(1028, 768, padding_idx=3)
  (layers): ModuleList(
    (0): BartDecoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (encoder_attn): BartAttention(
        (k_proj): Linear(in_features=768, out_features=768, bias=True)
        (v_proj): Linear(in_features=768, out_features=768, bias=True)
        (q_proj): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
      )
      (encoder_attn_layer_norm): LayerNorm((768,), eps=1e-05

In [45]:
generate_out = model.generate(inputs['input_ids'], num_beams=16, min_length=30, max_length=160, early_stopping=True)
tokenizer.decode(generate_out[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

'한국 전쟁 직후 현대건설은 전쟁으로 파괴된 도시와 교량, 도로, 집, 도로, 집, 건물 등을 복구하면서 점차 늘어가는 건설 수요로 승승장구하게 되었하였다. 그 뒤에도 늘어나는 건설 수요 등을 감안하여 그는 시멘트 공장 설립을 추진, 1964년 6월 현대 시멘트공장을 준공하여 시멘트도 자체적으로 조달하였다. 그 뒤 낙동강 고령교 복구, 한강 인도교 복구, 한강 인도교 복구, 제1한강교 복구, 제1한강교 복구, 인천 제1도크 복구 등의 사업을 수주하여 1960년에는 국내 건설업체중 도급도크 복구 등의 사업을 수주하여 1960년에는 국내 건설업체중 도급한도액이 1위를 차지하게 되었다. 1964년 단양에 시멘트 공장을 완공하였으며, 1965년에는 국내 최초로'